In [17]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
import tensorflow as tf
import numpy as np
!pip install tflearn
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
stemmer=LancasterStemmer()

In [19]:
from google.colab import files
files.upload()

Saving intents.json to intents (1).json


{'intents.json': b'{"intents": [\r\n    {"tag": "greeting",\r\n     "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day"],\r\n     "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],\r\n     "context_set": ""\r\n    },\r\n    {"tag": "goodbye",\r\n     "patterns": ["Bye", "See you later", "Goodbye"],\r\n     "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]\r\n    },\r\n    {"tag": "thanks",\r\n     "patterns": ["Thanks", "Thank you", "That\'s helpful"],\r\n     "responses": ["Happy to help!", "Any time!", "My pleasure"]\r\n    },\r\n    {"tag": "hours",\r\n     "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],\r\n     "responses": ["We\'re open every day 9am-9pm", "Our hours are 9am-9pm every day"]\r\n    },\r\n    {"tag": "location",\r\n     "patterns": ["What is your location?", "Where are you located?", "What is your address?"

In [21]:
with open('intents.json') as json_data:
  intent=json.load(json_data)

In [22]:
intent

{'intents': [{'context_set': '',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.'],
   'tag': 'goodbye'},
  {'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'tag': 'thanks'},
  {'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day'],
   'tag': 'hours'},
  {'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of London Alley and Bridge A

In [23]:
words = []
documents = []
classes = []
ignore = ['?']
for intentions in intent['intents']:
  for pattern in intentions['patterns']:
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w, intentions['tag']))
    if intentions['tag'] not in classes:
      classes.append(intentions['tag'])


In [24]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))


In [25]:
training = []
output = []
output_empty = [0]*len(classes)
for doc in documents:
  bag = []
  pattern_words = doc[0]
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
  for w in words:
   bag.append(1) if w in pattern_words else bag.append(0)
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1
  training.append([bag,output_row])
random.shuffle(training)
training = np.array(training)
train_X=list(training[:,0])
train_y=list(training[:,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [33]:
import tflearn
from tensorflow.python.framework import ops
ops.reset_default_graph()
import tflearn
net=tflearn.input_data(shape=[None,len(train_X[0])])
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,len(train_y[0]),activation='softmax')
net=tflearn.regression(net)
model=tflearn.DNN(net,tensorboard_dir='tflearn_logs')
model.fit(train_X,train_y,n_epoch=1000,batch_size=8,show_metric=True)
model.save('model.tflearn')


Training Step: 3999  | total loss: 0.01258 | time: 0.009s
| Adam | epoch: 1000 | loss: 0.01258 - acc: 0.9998 -- iter: 24/31
Training Step: 4000  | total loss: 0.01270 | time: 0.011s
| Adam | epoch: 1000 | loss: 0.01270 - acc: 0.9999 -- iter: 31/31
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [34]:
import pickle
pickle.dump({'words':words,'classes':classes,'train_X':train_X,'train_y':train_y},open("training_data","wb"))

In [35]:
data=pickle.load(open("training_data","rb"))
words = data['words']
classes = data['classes']
train_X = data['train_X']
train_y = data['train_y']

In [41]:
with open('intents.json') as json_data:
 intent = json.load(json_data)

In [42]:
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [46]:
def clean_up_sentence(sentence):
  sentence_words=nltk.word_tokenize(sentence)
  sentence_words=[stemmer.stem(words.lower()) for words in sentence_words]
  return sentence_words

def bow(sentence,words,show_details=False):
    sentence_words=clean_up_sentence(sentence)
    bag=[0]*len(words)
    for s in sentence_words:
      for i,w in enumerate(words):
        if w==s:
          bag[i]=1
          if show_details:
             print("found in bag: %s"%w)
    return(np.array(bag))


In [51]:
ERROR_THRESHOLD=0.30
def classify(sentence):
  results=model.predict([bow(sentence,words)])[0]
  results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append((classes[r[0]], r[1]))
    return return_list
def response(sentence,userID='123',show_details='False'):
  results=classify(sentence)
  if results:
    while results:
      for i in intent['intents']:
        if i['tag'] == results[0][0]:
          return print(random.choice(i['responses']))

        results.pop(0)



In [54]:
response('What is menu for today?')

IndexError: ignored